# 사용자 행동 인식 예측 분류

* 30명에게 스마트폰 센서를 장착한 뒤 사람의 동작과 관련된 여러 가지 피처를 수집한 데이터 세트 
* 결정 트리 ( DecisionTreeClassifier ) 를 이용하여 예측 분류 실습 <br><br>

* dataset
    * features 
        * 총 561개의 피처명으로 구성된 데이터셋
    * activity_labels 
        * 6 개의 레이블 명으로 구성된 데이터셋 
    * X_test , y_test
    * X_train , y_train
        * 데이터가 이미 학습용과 테스트용으로 분리 되어져 있음
        
먼저, features.txt 파일부터 살펴보자.

In [1]:
import pandas as pd 

feature_name_df = pd.read_csv("UCI_HAR_Dataset/features.txt",sep='\s+',header=None, names=['column_index','column_name'])

print(feature_name_df.shape)
print(feature_name_df)
feature_name_df.iloc[:,1][:5]

(561, 2)
     column_index                           column_name
0               1                     tBodyAcc-mean()-X
1               2                     tBodyAcc-mean()-Y
2               3                     tBodyAcc-mean()-Z
3               4                      tBodyAcc-std()-X
4               5                      tBodyAcc-std()-Y
..            ...                                   ...
556           557      angle(tBodyGyroMean,gravityMean)
557           558  angle(tBodyGyroJerkMean,gravityMean)
558           559                  angle(X,gravityMean)
559           560                  angle(Y,gravityMean)
560           561                  angle(Z,gravityMean)

[561 rows x 2 columns]


0    tBodyAcc-mean()-X
1    tBodyAcc-mean()-Y
2    tBodyAcc-mean()-Z
3     tBodyAcc-std()-X
4     tBodyAcc-std()-Y
Name: column_name, dtype: object

feature 의 수가 561 개로 너무 많다. 하나씩 다 살펴볼 수 없으며 추후 오류를 방지하기 위해 중복되는 피처명이 있나 확인해 보자.

In [2]:
feature_dup_df = feature_name_df.groupby('column_name').count()
print(feature_dup_df[feature_dup_df['column_index']>1].count())
feature_dup_df[feature_dup_df['column_index']>1].head(5)

column_index    42
dtype: int64


,column_index
column_name,
"fBodyAcc-bandsEnergy()-1,16",3
"fBodyAcc-bandsEnergy()-1,24",3
"fBodyAcc-bandsEnergy()-1,8",3
"fBodyAcc-bandsEnergy()-17,24",3
"fBodyAcc-bandsEnergy()-17,32",3


총 42개의 피처가 중복되고 있으므로 _ 1 , _ 2 을 추가하여 중복되는 피처들에게 새로운 피처명을 부여한 Dataframe 을 반환하는 함수를 만들어보자. <br>
### 피처명 중복 처리 함수

* count 는 해당 컬럼 값들 중 동일한 값의 갯수 반환 , cumcount 는 해당 컬럼 값들 중 중복된 값의 중복 횟수 즉, 원본 1개를 제외한 횟수 반환<br>
* column_index 와 column_name 으로 구성된 old_feature_name_df 과 column_name , dup_cnt ( 중복된 횟수 ) 로 구성된 feature_dup_df 를 merge 즉, 공통되는 column_name 기준으로 모든 행들에 대해 column_index 와 dup_cnt 를 매칭 시킨다. 중복되었던 column_name 이 다시 생긴 형태
<br>
* apply 와 lambda 로 원본 이후로 중복된 피처들에 _ 1, _ 2 ... 로 새로운 피처명 부여에서 Dataframe 에 반영 

In [3]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),columns=['dup_cnt']) 
    feature_dup_df = feature_dup_df.reset_index() # column name 으로 되어 있는 index reset 
    
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(),feature_dup_df,how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name','dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1])
                                                                                             if x[1]>0 else x[0],axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'],axis=1)
    return new_feature_name_df
    

### 데이터 전처리 함수 

내부에서 피처명 중복 처리 함수를 호출하며 반환 값 처리, feature , label 학습과 테스트를 위한 dataset 을 반환. 

In [6]:
def get_human_activity_recognition_dataset():
    feature_name_df = pd.read_csv("UCI_HAR_Dataset/features.txt",sep='\s+',header=None, names=['column_index','column_name'])
    # 컬럼명 중복 처리
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # 전처리된 feature 들을 학습, 테스트 dataset 의 feature 로 넣어줘야 해서 list 로 변환
    feature_name = new_feature_name_df.iloc[:,1].values.tolist()
    
    X_train = pd.read_csv("UCI_HAR_Dataset/train/X_train.txt",sep='\s+',header=None, names=feature_name)
    X_test = pd.read_csv("UCI_HAR_Dataset/test/X_test.txt",sep='\s+',header=None, names=feature_name)
    
    y_train = pd.read_csv("UCI_HAR_Dataset/train/y_train.txt",sep='\s+',header=None, names=['action'])
    y_test = pd.read_csv("UCI_HAR_Dataset/test/y_test.txt",sep='\s+',header=None, names=['action'])
    
    return X_train , X_test, y_train, y_test

X_train, X_test, y_train , y_test = get_human_activity_recognition_dataset()

받아온 학습용 데이터가 그대로 예측 모델 학습을 진행해도 되는지 확인 해보자.

In [15]:
print(X_train.info(),'\n\n')
y_train.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 561 entries, tBodyAcc-mean()-X to angle(Z,gravityMean)
dtypes: float64(561)
memory usage: 31.5 MB
None 




action
6         1407
5         1374
4         1286
1         1226
2         1073
3          986
dtype: int64

X_train 은 전부 float 값이기 때문에 따로 인코딩이 필요없고, y_train 의 경우는 label 이 골고루 분포 되어 있다.<br>
이제 DecisionTreeClassifier 를 이용해서 예측 분류를 진행해보자. <br>
먼저, 하이퍼 파라미터를 조정해주지 않았을 때의 기본 상태 예측 정확도를 보자.

In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt_clf = DecisionTreeClassifier(random_state = 77)
dt_clf.fit(X_train,y_train)
pred = dt_clf.predict(X_test)
accuracy = accuracy_score(y_test,pred)
print('결정 트리 예측 정확도: {0:.4f}'.format(accuracy),'\n')

print('DecisionTreeClassifier 기본 하이퍼 파라미터: \n',dt_clf.get_params())

결정 트리 예측 정확도: 0.8595 

DecisionTreeClassifier 기본 하이퍼 파라미터: 
 {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'presort': 'deprecated', 'random_state': 77, 'splitter': 'best'}


지난번에 배웠던 GridSearchCV 를 이용해서 하이퍼 파라미터를 조절하여 결정 트리의 depth 가 실제로 예측 정확도에 영향을 주는지 확인한다.<br>
( GridSearchCV 는 p.113 부터 참고 )

In [33]:
from sklearn.model_selection import GridSearchCV

params = {'max_depth':[6,8,10,12,16,20,24]}

gridcv = GridSearchCV(dt_clf,param_grid=params,scoring='accuracy',cv=5,verbose=0)
gridcv.fit(X_train,y_train)
print('GridSearchCV 최고 평균 정확도 수치 : {0:.4f}'.format(gridcv.best_score_))
print('GridSearchCV 최적 하이퍼 파라미터 : ',gridcv.best_params_)

GridSearchCV 최고 평균 정확도 수치 : 0.8520
GridSearchCV 최적 하이퍼 파라미터 :  {'max_depth': 8}


5 개 CV세트에 각 하이퍼 파라미터를 순차적으로 적용하였을 때의 각 검증용 데이터 세트의 평균 정확도 수치를 모두 알고 싶다면  cv_results_ 를 사용한다.

In [35]:
cv_results_df = pd.DataFrame(gridcv.cv_results_)
cv_results_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.256004,0.009652,0.005005,1.850472e-06,6,{'max_depth': 6},0.813052,0.868797,0.817687,0.862585,0.870748,0.846574,0.025662,7
1,1.600684,0.056488,0.005006,8.869684e-07,8,{'max_depth': 8},0.819850,0.834126,0.857823,0.858503,0.889796,0.852020,0.023907,1
2,1.853689,0.050931,0.004805,3.990906e-04,10,{'max_depth': 10},0.789939,0.818491,0.857143,0.887075,0.889796,0.848489,0.038982,4
3,2.080981,0.099250,0.004606,4.893568e-04,12,{'max_depth': 12},0.796057,0.813732,0.863265,0.889796,0.893197,0.851210,0.039607,2
4,2.322516,0.178142,0.004807,4.001656e-04,16,{'max_depth': 16},0.801496,0.821890,0.859864,0.880272,0.889796,0.850663,0.033876,3
5,2.348405,0.213638,0.004768,3.877832e-04,20,{'max_depth': 20},0.798776,0.821890,0.859184,0.870068,0.889116,0.847807,0.032885,6
6,2.350773,0.212111,0.004802,4.004047e-04,24,{'max_depth': 24},0.799456,0.821890,0.859184,0.870068,0.889116,0.847943,0.032683,5


위 표를 보면 max_depth 가 8 일 때 최고 정확도를 보인다.<br><br>
즉, 결정 트리는 더 완벽한 규칙을 학습 데이터 세트에 적용하기 위해 노드를 지속적으로 분할하면 깊이가 깊어지고, 더 복잡한 모델이 되면서 정확도 수치 일부 떨어지게 된다.<br><br>
<span style="color:red">#유의 : 위 표는 학습 시킨 모델에 대한 테스트 데이터 정확도가 아니라 검증용 데이터셋에 대한 예측 정확도</span><br><br>
이번에는 테스트 데이터셋 에서 하이퍼 파라미터를 조정한 결정 트리의 예측 정확도가 어떻게 나타나는지 확인해보자.


In [40]:
max_depths = [6,8,10,12,16,20,24]

for depth in max_depths:
    dt_clf = DecisionTreeClassifier(max_depth=depth,random_state=16)
    dt_clf.fit(X_train,y_train)
    pred = dt_clf.predict(X_test)
    accuracy = accuracy_score(y_test,pred)
    print('max depth = {0}, 정확도 = {1:.4f}'.format(depth,accuracy))

max depth = 6, 정확도 = 0.8537
max depth = 8, 정확도 = 0.8700
max depth = 10, 정확도 = 0.8694
max depth = 12, 정확도 = 0.8653
max depth = 16, 정확도 = 0.8599
max depth = 20, 정확도 = 0.8592
max depth = 24, 정확도 = 0.8592


테스트 데이터셋에서도 GridSearchCV 예제와 마찬가지로 깊이가 깊어질수록 과적합의 영향력이 커지면서 정확도가 떨어지고 있다.<br>
즉, 복잡한 모델보다도 트리 깊이를 낮춘 단순한 모델이 더욱 효과적일 수 있음을 알 수 있다.<br><br>
그렇다면 max_depth 외에 다른 하이퍼 파라미터도 조정을 해 볼 것이다.

In [44]:
params = {'max_depth':[8,12,16,20],
         'min_samples_split':[16,24]
         }

gridcv = GridSearchCV(dt_clf,param_grid=params,scoring='accuracy',cv=5,verbose=0)
gridcv.fit(X_train,y_train)
print('GridSearchCV 최고 평균 정확도 수치 : {0:.4f}'.format(gridcv.best_score_))
print('GridSearchCV 최적 하이퍼 파라미터 : ',gridcv.best_params_)

GridSearchCV 최고 평균 정확도 수치 : 0.8535
GridSearchCV 최적 하이퍼 파라미터 :  {'max_depth': 8, 'min_samples_split': 24}


GridSearchCV 에서 refit=True 가 default 이기 때문에 자동으로 최적의 하이퍼 파라미터로 estimator 를 학습해서 <br>best_estimator_ 로 저장한다. = Estimator 객체

In [46]:
best_dt_clf = gridcv.best_estimator_
best_pred = best_dt_clf.predict(X_test)
accuracy = accuracy_score(best_pred,y_test)
print('최적 파라미터 결정 트리 예측 정확도 : {0:.4f}'.format(accuracy))

최적 파라미터 결정 트리 예측 정확도 : 0.8694


#### 마지막으로 피처의 중요도 파악

In [98]:
import_feature_df = pd.DataFrame(best_dt_clf.feature_importances_,index=X_train.columns)

import_feature_df.columns = ['feature_importance']

import_feature_df =import_feature_df[import_feature_df['feature_importance']!= 0]
import_feature_df = import_feature_df.sort_values(by=['feature_importance'],ascending=False).iloc[:10]

import_feature_df

,feature_importance
tGravityAcc-min()-X,0.254008
fBodyAcc-mad()-X,0.213132
"angle(Y,gravityMean)",0.143406
fBodyAccMag-energy(),0.115776
"tGravityAcc-arCoeff()-Z,2",0.101810
fBodyGyro-maxInds-X,0.024033
"tBodyGyro-correlation()-Y,Z",0.016556
tGravityAcc-energy()-Y,0.016476
tGravityAccMag-arCoeff()1,0.015782
fBodyAcc-max()-Y,0.008847


In [92]:
# iplot
import plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

import_feature_df.iplot(kind='barh')